In [1]:
import pandas as pd
import glob
import os

In [2]:
# http://www.football-data.co.uk
SOURCE_FOLDER_FILES_PATTERN = "../knowledge/data/source/*.csv"
FINAL_SET_FOLDER = "../knowledge/data"
FINAL_SET_NAME = 'football_stats.csv'
FINAL_SET_FULL_PATH = os.path.join(FINAL_SET_FOLDER, FINAL_SET_NAME)

In [4]:
files = glob.glob(SOURCE_FOLDER_FILES_PATTERN)
# print(files)

In [5]:
SELECTED_COLUMNS = ['Div', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'] #, 'Date', 'HTHG','HTAG', 'HTR'

In [6]:
football = pd.DataFrame(columns=SELECTED_COLUMNS, index = None)
for file in files:
    football = football.append(pd.read_csv(file)[SELECTED_COLUMNS])

In [7]:
football.head()

,Div,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,F1,Monaco,Toulouse,3,2,H
1,F1,Lyon,Strasbourg,4,0,H
2,F1,Metz,Guingamp,1,3,A
3,F1,Montpellier,Caen,1,0,H
4,F1,Paris SG,Amiens,2,0,H


In [8]:
football["Div"].value_counts()

F1     140
I1     138
E0     130
SP1    130
D1     117
Name: Div, dtype: int64

In [15]:
football["FTR"].value_counts()

H    296
A    206
D    153
Name: FTR, dtype: int64

In [22]:
RESULTS_COLUMNS_MAPPING = {
    "HomeTeam": {
        "FTR_A": "HL",
        "FTR_H": "HW",
        "FTR_D": "HD"
    },
    "AwayTeam": {
        "FTR_A": "AW",
        "FTR_H": "AL",
        "FTR_D": "AD"        
    }
}

COUNTRY_DIVISION_MAPPING = {
    "E0": "England",
    "D1": "Germany",
    "F1": "France",
    "SP1": "Spain",
    "I1": "Italy"
}

def get_results(df, home_away_team):
    df_grouped = pd.get_dummies(football, columns = ['FTR']).groupby([home_away_team]).sum()
    return df_grouped.rename(RESULTS_COLUMNS_MAPPING[home_away_team], axis="columns")
        
results = get_results(football, "HomeTeam").\
      merge(get_results(football, "AwayTeam"),left_index = True, right_index = True).reset_index().\
      rename({"HomeTeam":"team"}, axis="columns").\
      merge(football[["Div", "HomeTeam"]].drop_duplicates().rename({"HomeTeam":"team"}, axis="columns"),
            on = ['team'])

results["W"] = results["HW"] + results["AW"]
results["L"] = results["HL"] + results["AL"]
results["D"] = results["HD"] + results["AD"]
results["points"] = results["W"] * 3 + results["D"] *1
results["home_points"] = results["HW"] * 3 + results["HD"] *1
results["away_points"] = results["AW"] * 3 + results["AD"] *1
results["country"] = results["Div"].map(COUNTRY_DIVISION_MAPPING)

results.sort_values('Div',inplace = True)

results.to_csv(FINAL_SET_FULL_PATH, index = False)
results["Div"].map(COUNTRY_DIVISION_MAPPING).value_counts()

Italy      20
France     20
Spain      20
England    20
Germany    18
Name: Div, dtype: int64

In [11]:
def make_entity(df, column_name):
    for x in list(df[column_name].unique()):
        print('"{}","{}"'.format(x,x))

In [13]:
make_entity(results, "team")

"Wolfsburg","Wolfsburg"
"Ein Frankfurt","Ein Frankfurt"
"Hamburg","Hamburg"
"Dortmund","Dortmund"
"Hannover","Hannover"
"Hertha","Hertha"
"Hoffenheim","Hoffenheim"
"Leverkusen","Leverkusen"
"M'gladbach","M'gladbach"
"Mainz","Mainz"
"Freiburg","Freiburg"
"RB Leipzig","RB Leipzig"
"FC Koln","FC Koln"
"Werder Bremen","Werder Bremen"
"Bayern Munich","Bayern Munich"
"Augsburg","Augsburg"
"Schalke 04","Schalke 04"
"Stuttgart","Stuttgart"
"Newcastle","Newcastle"
"West Brom","West Brom"
"Watford","Watford"
"Tottenham","Tottenham"
"Swansea","Swansea"
"Huddersfield","Huddersfield"
"Liverpool","Liverpool"
"Stoke","Stoke"
"Man City","Man City"
"Southampton","Southampton"
"West Ham","West Ham"
"Man United","Man United"
"Leicester","Leicester"
"Crystal Palace","Crystal Palace"
"Burnley","Burnley"
"Brighton","Brighton"
"Bournemouth","Bournemouth"
"Chelsea","Chelsea"
"Arsenal","Arsenal"
"Everton","Everton"
"Monaco","Monaco"
"Metz","Metz"
"Marseille","Marseille"
"Bordeaux","Bordeaux"
"Rennes","Rennes"


In [ ]:
make_entity(football, "HomeTeam")